In [ ]:
from transformers import get_cosine_schedule_with_warmup
from torch_optimizer import Ranger
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from multiprocessing import cpu_count
from os import environ
from platform import system
from utilities.utils import *

environ["TOKENIZERS_PARALLELISM"] = "false"
pl.seed_everything(seed=101)

In [ ]:
LEARNING_RATE = 2.5e-2
BATCH_SIZE = 128
WEIGHT_DECAY = 1e-2
EPOCHS = 25
MAX_LEN = None
CELL_TYPE = "gru"
N_JOBS = cpu_count() if system() != "Windows" else 0

In [ ]:
class SEQ(pl.LightningModule):
    def __init__(self,
                 input_dim,
                 cell_dim=128,
                 embed_dim=128,
                 num_layers=3,
                 dropout=0.1,
                 cell_type=CELL_TYPE,
                 bidirectional=True,
                 use_scheduler=True,
                 train_dataset=None,
                 val_dataset=None,
                 test_dataset=None):

        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=input_dim,
                                      embedding_dim=embed_dim,
                                      padding_idx=0)
        

        c = (2 if bidirectional else 1)
        k = (2 if cell_type == "lstm" else 1)
        if cell_type == "lstm":
            self.cell = nn.LSTM(input_size=embed_dim, 
                                hidden_size=cell_dim, 
                                dropout=dropout,
                                batch_first=True,
                                num_layers=num_layers, 
                                bidirectional=bidirectional)
        else:
            self.cell = nn.GRU(input_size=embed_dim, 
                               hidden_size=cell_dim, 
                               dropout=dropout,
                               batch_first=True,
                               num_layers=num_layers, 
                               bidirectional=bidirectional)
            
        fc_dim = k * c * num_layers * cell_dim
        self.fc = nn.Linear(fc_dim, 2)
        self.dropout = nn.Dropout(p=dropout)
        self.use_scheduler = use_scheduler
        self.loss_fn = nn.NLLLoss()
        self.cell_type = cell_type
        self.adaptive_avg_pool = nn.AdaptiveAvgPool1d(cell_dim)
        ## Datasets ##
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        ## metrics ##
        self.train_f1score = torchmetrics.F1Score()
        self.val_f1score = torchmetrics.F1Score()
        self.test_f1score = torchmetrics.F1Score()
        ## Hyperparameters ##
        self.batch_size = BATCH_SIZE
        self.learning_rate = LEARNING_RATE
        self.weight_decay = WEIGHT_DECAY
        if self.use_scheduler:
            self.total_steps = len(train_dataset) // self.batch_size


    def train_dataloader(self):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size,
                          shuffle=True,
                          num_workers=N_JOBS,
                          drop_last=False)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)


    def forward(self, input_ids):
        _, out = self.cell(self.embedding(input_ids))
        if self.cell_type == "lstm": torch.cat(out, dim=-1)
        out = torch.reshape(out, (out.size(1), -1))
        out = F.relu(out)
        out = self.dropout(out)
        out = self.fc(out)
        out = F.log_softmax(out, dim=-1)
        return out
    
    
    def _shared_evaluation_step(self, batch, batch_idx):
        ids, lbls = batch
        out = self(ids)
        loss = self.loss_fn(out, lbls)
        preds = torch.argmax(out, dim=-1)
        return loss, preds


    def training_step(self, batch, batch_idx):
        loss, preds = self._shared_evaluation_step(batch, batch_idx)
        self.train_f1score(preds, batch[-1])
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_f1score", self.train_f1score, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    
    def validation_step(self, batch, batch_idx):
        loss, preds = self._shared_evaluation_step(batch, batch_idx)
        self.val_f1score(preds, batch[-1])
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_f1score", self.val_f1score, on_step=False, on_epoch=True, prog_bar=True)

    
    def test_step(self, batch, batch_idx):
        loss, preds = self._shared_evaluation_step(batch, batch_idx)
        self.test_f1score(preds, batch[-1])
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", self.test_f1score, on_step=False, on_epoch=True, prog_bar=True)


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        ids, _ = batch
        return torch.argmax(self(ids), dim=-1)
    
    
    def configure_optimizers(self):
        optimizer = Ranger(self.parameters(),
                           lr=self.learning_rate,
                           weight_decay=self.weight_decay)

        if self.use_scheduler:
            scheduler = get_cosine_schedule_with_warmup(optimizer=optimizer,
                                                        num_warmup_steps=1,
                                                        num_training_steps=self.total_steps)
            lr_scheduler = {
                'scheduler': scheduler, 
                'interval': 'epoch', 
                'frequency': 1
            }
            return [optimizer], [lr_scheduler]
        else:
            return [optimizer]

In [ ]:
with open("../data/full_vocab_sug_tree_bank_tokenier.txt", mode="r", encoding="utf-8") as f:
    vocab = [s.strip() for s in f.readlines()]
    VOCAB2IDX = {v:k for (k, v) in enumerate(vocab)}

In [ ]:
input_ids, labels = get_encoded_input("../data/App_Training.csv", 
                                      maxlen=MAX_LEN, 
                                      vocab2idx=VOCAB2IDX)

dataset = TensorDataset(torch.LongTensor(input_ids),
                        torch.LongTensor(labels))

L = len(labels)

train_sz, val_sz = L-int(0.1*L), int(0.1*L)
train_dataset, val_dataset = random_split(dataset, (train_sz, val_sz))                                                                                                    

In [ ]:
input_ids, labels = get_encoded_input("../data/App_Test_Labeled.csv", 
                                      maxlen=MAX_LEN, 
                                      vocab2idx=VOCAB2IDX)

test_dataset = TensorDataset(torch.LongTensor(input_ids),
                             torch.LongTensor(labels))                                                                                              

In [ ]:
model = SEQ(input_dim=len(VOCAB2IDX),
            cell_type=CELL_TYPE,
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            test_dataset=test_dataset,
            use_scheduler=True)

earlystopping_callback = EarlyStopping(monitor="val_f1score", 
                                       min_delta=1e-4, 
                                       patience=10, 
                                       mode="max")

checkpoint_callback = ModelCheckpoint(dirpath="./saved_weights",
                                      filename=f"seq-{CELL_TYPE}-sug",
                                      save_top_k=1, 
                                      mode="max",
                                      monitor="val_f1score",
                                      save_weights_only=True)

trainer = pl.Trainer(accelerator="gpu",
                     max_epochs=EPOCHS,
                     precision=16,
                     log_every_n_steps=1,
                     callbacks=[earlystopping_callback, 
                                checkpoint_callback])

In [ ]:
trainer.fit(model)

In [ ]:
model.load_state_dict(torch.load(f"./saved_weights/seq-{CELL_TYPE}-sug.ckpt")["state_dict"])
trainer.test(model)